In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

class GraphState(TypedDict):
    messages: list[str]
    topic: str
    answer: str

builder = StateGraph(GraphState)

def dummy_node(state: GraphState) -> GraphState:
    state["answer"] = "Hello from LangGraph"
    return state

builder.add_node("dummy", dummy_node)
builder.add_edge(START, "dummy")
builder.add_edge("dummy", END)

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)


In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

# add_messages merges message lists across graph nodes
# (equivalent to left + right, but safer for LangGraph)
# def add_messages(left: list, right: list):
#     return left + right

# Define the graph state schema
class State(TypedDict):
    # messages will be automatically appended using add_messages
    messages: Annotated[list, add_messages]


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

# -----------------------------
# 1️⃣ Define the graph state
# -----------------------------
class State(TypedDict):
    # messages will be automatically merged across nodes using add_messages
    messages: Annotated[list, add_messages]

# -----------------------------
# 2️⃣ Define a tool
# -----------------------------
@tool
def search(query: str):
    """Simulate a web search."""
    # Placeholder search logic
    return [f"Search result for: '{query}'"]

# Register tools
tools = [search]

# -----------------------------
# 3️⃣ Create a ToolNode
# -----------------------------
tool_node = ToolNode(tools)

# -----------------------------
# 4️⃣ Build the stateful graph
# -----------------------------
builder = StateGraph(State)
builder.add_node(tool_node)  # add the ToolNode to the graph
graph = builder.compile()

# -----------------------------
# 5️⃣ Run the graph
# -----------------------------
# Initial state
state = {"messages": ["Welcome to LangGraph!"]}

# Example query
query_input = {"query": "What is LangGraph?"}

# Execute the tool node within the graph
new_state = graph.run(state=state, inputs=query_input)

# Print aggregated messages
print(new_state["messages"])


In [ ]:
from langchain_openai import ChatOpenAI

# We will set streaming=True so that we can stream tokens
# See the streaming section for more information on this.
model = ChatOpenAI(temperature=0, streaming=True)

In [ ]:
bound_model = model.bind_tools(tools)

In [ ]:
# Define the function that determines whether to continue or not
from typing import Literal


def should_continue(state: State) -> Literal["action", "__end__"]:
    """Return the next node to execute."""
    last_message = state["messages"][-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "__end__"
    # Otherwise if there is, we continue
    return "action"


# Define the function that calls the model
def call_model(state: State):
    response = model.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": response}

In [ ]:
from langgraph.graph import StateGraph, START

# Define a new graph
workflow = StateGraph(State)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

In [ ]:
input_message = HumanMessage(content="what is my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

In [ ]:
input_message = HumanMessage(content="what is my name?")
for event in app.stream(
    {"messages": [input_message]},
    {"configurable": {"thread_id": "3"}},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

In [ ]:
input_message = HumanMessage(content="You forgot??")
for event in app.stream(
    {"messages": [input_message]},
    {"configurable": {"thread_id": "2"}},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()